In [14]:
from io import open
import os
import unicodedata
import string
import re
import random
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import warnings

warnings.filterwarnings("ignore")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# from google.colab import drive
# drive.mount('/content/drive')

In [15]:
SOS_token = 0
EOS_token = 1

class LanguageVocabulary(object):
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2

    def add_sentence(self, sentence):
        for word in sentence.split(' '):
            self.add_word(word)


    def add_word(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [17]:
class Preprocessing:

    # DATA_DIR = '/content/drive/My Drive/Colab Notebooks/translation_data/'
    DATA_DIR = './translation_data/'

    def __init__(self, lang, reverse=False):
        self.lang = lang
        self.source_path = os.path.join(self.DATA_DIR, lang + '.txt')
        self.target_path = os.path.join(self.DATA_DIR, 'eng-' + lang + '.txt')
        self.reverse = reverse

    @staticmethod
    def unicode_to_ascii(s):
        return ''.join(
            c for c in unicodedata.normalize('NFD', s)
            if unicodedata.category(c) != 'Mn'
        )

    def normalize_string(self, s):
        s = self.unicode_to_ascii(s.lower().strip())
        s = re.sub(r"([.!?])", r" \1", s)
        s = re.sub(r"[^a-zA-Zа-яА-Яє-їЄ-Ї.!?]+", r" ", s)
        # s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
        return s

    def prepare_file(self, source_path, target_path):
        with open(source_path, encoding='utf-8') as file1, open(target_path, 'w', encoding='utf-8') as file2:
            for line in file1:
                file2.write(line.strip().split('\t')[0] + '\t' + line.strip().split('\t')[1] + '\n')

    def prepare_data(self, lang1, lang2, reverse=False):
        input_lang, output_lang, pairs = self.read_languages(lang1, lang2, reverse)
        print("Read %s sentence pairs" % len(pairs))
        print("Counting words...")
        for pair in pairs:
            input_lang.add_sentence(pair[0])
            output_lang.add_sentence(pair[1])
        print("Counted words:")
        print(input_lang.name, input_lang.n_words)
        print(output_lang.name, output_lang.n_words)
        return input_lang, output_lang, pairs

    def read_languages(self, lang1, lang2, reverse=False):
        print("Reading lines...")
        path = os.path.join(self.DATA_DIR, '%s-%s.txt' % (lang1, lang2))
        lines = open(path, encoding='utf-8').read().strip().split('\n')
        pairs = [[self.normalize_string(s) for s in l.split('\t')] for l in lines]
        if reverse:
            pairs = [list(reversed(p)) for p in pairs]
            input_lang = LanguageVocabulary(lang2)
            output_lang = LanguageVocabulary(lang1)
        else:
            input_lang = LanguageVocabulary(lang1)
            output_lang = LanguageVocabulary(lang2)
        return input_lang, output_lang, pairs

    def get_data(self):
        self.prepare_file(self.source_path, self.target_path)
        input_lang, output_lang, pairs = self.prepare_data('eng', self.lang, self.reverse)
        return input_lang, output_lang, pairs

input_lang, output_lang, pairs = Preprocessing(lang='ukr').get_data()
print(random.choice(pairs))

Reading lines...
Read 156173 sentence pairs
Counting words...
Counted words:
eng 10085
ukr 30616
['i don t trust you .', 'я тобі не довіряю .']


In [18]:
def get_max_length(pairs):
  max_length = 0
  for pair in pairs:
    max_in_pair = max(len(pair[0]), len(pair[1]))
    if max_in_pair > max_length:
      max_length = max_in_pair
  return max_length
MAX_LENGTH = get_max_length(pairs)

In [19]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [20]:
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]

def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)

def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

In [21]:
teacher_forcing_ratio = 0.5

def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):

    encoder_hidden = encoder.initHidden()
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()
    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)
    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)
    loss = 0
    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)
    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False
    if use_teacher_forcing:
        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing
    else:
        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()
            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break
    loss.backward()
    encoder_optimizer.step()
    decoder_optimizer.step()
    return loss.item() / target_length

In [ ]:
import time
import math

def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / percent
    rs = es - s
    return '%s (- eta: %s)' % (asMinutes(s), asMinutes(rs))

In [ ]:
def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)

    training_pairs = [tensorsFromPair(random.choice(pairs)) for i in range(n_iters)]

    criterion = nn.NLLLoss()

    for epoch in range(1, n_iters + 1):
        training_pair = training_pairs[epoch - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if epoch % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, epoch / n_iters),
                                         epoch, epoch / n_iters * 100, print_loss_avg))

        if epoch % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0
    showPlot(plot_losses)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
plt.switch_backend('agg')


def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

In [22]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()
        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for i in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[i], encoder_hidden)
            encoder_outputs[i] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)
        decoder_hidden = encoder_hidden
        decoded_words = []

        for di in range(max_length):
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])
            decoder_input = topi.squeeze().detach()
        return decoded_words

In [85]:
from torchtext.data.metrics import bleu_score

def evaluateRandomly(encoder, decoder, n=10):
    candidate_corpus = []
    references_corpus = []
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        references_corpus.append(pair[1].split(' '))
        output_words = evaluate(encoder, decoder, pair[0])
        candidate_corpus.append(output_words)
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')
    print('Blue score = ', bleu_score(candidate_corpus, references_corpus, max_n=1, weights=[1]))

In [25]:
hidden_size = 256

In [23]:
encoder1 = EncoderRNN(input_lang.n_words, hidden_size).to(device)
decoder1 = DecoderRNN(hidden_size, output_lang.n_words).to(device)
trainIters(encoder1, decoder1, 250000, print_every=5000)

1m 32s (- eta: 75m 49s) (5000 2%) 5.0252
2m 52s (- eta: 69m 2s) (10000 4%) 4.5566
4m 13s (- eta: 66m 8s) (15000 6%) 4.2489
5m 36s (- eta: 64m 24s) (20000 8%) 4.0173
6m 56s (- eta: 62m 31s) (25000 10%) 3.8550
8m 17s (- eta: 60m 48s) (30000 12%) 3.7310
9m 38s (- eta: 59m 13s) (35000 14%) 3.5877
10m 58s (- eta: 57m 39s) (40000 16%) 3.5022
12m 20s (- eta: 56m 11s) (45000 18%) 3.4080
13m 43s (- eta: 54m 52s) (50000 20%) 3.3255
15m 4s (- eta: 53m 27s) (55000 22%) 3.2612
16m 25s (- eta: 52m 0s) (60000 24%) 3.1990
17m 46s (- eta: 50m 35s) (65000 26%) 3.1125
19m 7s (- eta: 49m 11s) (70000 28%) 3.0998
20m 29s (- eta: 47m 49s) (75000 30%) 3.0050
21m 52s (- eta: 46m 29s) (80000 32%) 2.9912
23m 14s (- eta: 45m 7s) (85000 34%) 2.9808
24m 36s (- eta: 43m 44s) (90000 36%) 2.8970
25m 57s (- eta: 42m 20s) (95000 38%) 2.8491
27m 18s (- eta: 40m 57s) (100000 40%) 2.8135
28m 41s (- eta: 39m 37s) (105000 42%) 2.7844
30m 2s (- eta: 38m 14s) (110000 44%) 2.7551
31m 23s (- eta: 36m 51s) (115000 46%) 2.7401
32m

In [27]:
def save_model(model, path):
    torch.save(model.state_dict(), path)

def load_enc(path):
    model = EncoderRNN(input_lang.n_words, hidden_size).to(device)
    model.load_state_dict(torch.load(path, map_location=torch.device('cpu')))
    model.eval()
    return model

def load_dec(path):
    model = DecoderRNN(hidden_size, output_lang.n_words).to(device)
    model.load_state_dict(torch.load(path, map_location=torch.device('cpu')))
    model.eval()
    return model

# save_model(encoder1, 'encoder_ukr_eng_v3.pt')
# save_model(decoder1, 'decoder_ukr_eng_v3.pt')

In [87]:
encoder1 = load_enc('encoder_ukr_eng_v3.pt')
decoder1 = load_dec('decoder_ukr_eng_v3.pt')

evaluateRandomly(encoder1, decoder1)

> i walk quickly .
= я швидко ходжу .
< я швидко швидко . <EOS>

> tom pressed the wrong button .
= том натиснув не на ту кнопку .
< том натиснув цеи кнопку кнопку . <EOS>

> tom s wife had a baby girl yesterday .
= жінка тома вчора народила дівчинку .
< дружина тома тома дружина у дружина вчора . . <EOS>

> i failed the exam because i didn t study .
= я провалив екзамен бо я не вчився .
< я не був бо бо я я я . <EOS>

> tom s packing .
= том збирає валізи .
< том тома . <EOS>

> the movie is worth seeing at least two or three times .
= фільм вартии того щоб иого переглянути щонаименше двічі чи тричі .
< цеи фільм тут варто був у . . . <EOS>

> the question is do you know what you re doing .
= питання в тому чи ти знаєш що ти робиш .
< проблема це що що ви . . <EOS>

> i m going to the bank .
= я иду до банку .
< я іду до банку . <EOS>

> i don t get invited to parties anymore .
= мене більше не запрошують на вечірки .
< мене не не не чекати . <EOS>

> tom s smarter than mary .
= том р